## 3. Кодирование признаков

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel('data/data_ford_price.xlsx')
df.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [4]:
X = df.drop('price', axis=1)
y = df['price']

In [5]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']
 
for column in columns_to_change:
 print('Число уникальных значений признака {}: '.format(column), df[column].nunique())

Число уникальных значений признака cylinders:  6
Число уникальных значений признака title_status:  5
Число уникальных значений признака transmission:  3
Число уникальных значений признака drive:  3
Число уникальных значений признака size:  4


In [6]:
lb = LabelBinarizer()
education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']
lb.fit(education)
print('категории:', lb.classes_)
lb.transform(['нет', 'MSc'])

категории: ['BSc' 'MSc' 'PhD' 'начальное' 'нет' 'среднее']


array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

У класса LabelBinarizer, как и у двух остальных, есть атрибут classes_, который выводит список уникальных значений признака.

Итак, нам подходит однократное кодирование. Применим его к выбранным столбцам. Так как у нас нет отдельной тестовой выборки, то мы используем только один метод — fit_transform(). В качестве аргумента передаём таблицу с выбранными для преобразования признаками.

С помощью метода get_feature_names_out() получим список новых названий колонок:

In [7]:
one_hot_encoder = OneHotEncoder()

# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(df[columns_to_change]).toarray()
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
print(column_names)

['cylinders_3' 'cylinders_4' 'cylinders_5' 'cylinders_6' 'cylinders_8'
 'cylinders_10' 'title_status_clean' 'title_status_lien'
 'title_status_missing' 'title_status_rebuilt' 'title_status_salvage'
 'transmission_automatic' 'transmission_manual' 'transmission_other'
 'drive_4wd' 'drive_fwd' 'drive_rwd' 'drive_nan' 'size_compact'
 'size_full-size' 'size_mid-size' 'size_sub-compact' 'size_nan']


In [8]:
data_onehot = pd.DataFrame(data_onehot, index=df.index, columns=column_names)
data_onehot

,cylinders_3,cylinders_4,cylinders_5,cylinders_6,cylinders_8,cylinders_10,title_status_clean,title_status_lien,title_status_missing,title_status_rebuilt,...,transmission_other,drive_4wd,drive_fwd,drive_rwd,drive_nan,size_compact,size_full-size,size_mid-size,size_sub-compact,size_nan
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7013,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7014,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7015,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [9]:
data_new = pd.concat([df, data_onehot], axis=1)
data_new.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,...,transmission_other,drive_4wd,drive_fwd,drive_rwd,drive_nan,size_compact,size_full-size,size_mid-size,size_sub-compact,size_nan
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
data_new = data_new.drop(columns=columns_to_change)

In [11]:
data_new.shape

(7017, 30)

## 4. Обработка пропусков и выбросов

#### РАБОТА С ПРОПУСКАМИ

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


Символ ~ (тильда) означает, что мы выбираем все строки датасета data, где не выполняется условие data['weather'].isna(), то есть где нет пропусков в столбце 'weather'.

In [13]:
round(1- df[df['size'].isna()].shape[0] / df.shape[0], 2)

0.78

In [14]:
# удаляем все пропуски из датасета
X_c = X.dropna()

In [15]:
# отберем строки по индексам отфилтрованного датасета Х
y_c = y.iloc[X_c.index]

In [16]:
# проверяем размеры датасетов на равенство
X_c.shape[0] == y_c.shape[0]

True

In [17]:
X_c_train, X_c_valid, y_c_train, y_c_valid = train_test_split(X_c, y_c, test_size=0.2, random_state=30)

In [18]:
X_c_valid.shape

(1041, 11)

In [19]:
X_c_train.shape

(4161, 11)

In [20]:
def onehotcod(X_tr, X_t, y_tr, y_t, list_col):
    one_hot_encoder = OneHotEncoder()
    X_tr_ohe = one_hot_encoder.fit_transform(X_tr[list_col]).toarray()
    X_t_ohe = one_hot_encoder.transform(X_t[list_col]).toarray()
    
    columns = one_hot_encoder.get_feature_names_out(list_col)
    
    # создаем датасеты из массивов и заголовков из списка
    X_train_onehot_df = pd.DataFrame(X_tr_ohe, columns=columns)
    #print(X_train_onehot_df.shape[0])
    X_test_onehot_df = pd.DataFrame(X_t_ohe, columns=columns)
    #print(X_test_onehot_df.shape[0])
    
    # переустановим индексацию в таблицах, применив подряд сразу два метода:
    X_tr = X_tr.reset_index().drop(['index'], axis=1)
    X_t = X_t.reset_index().drop(['index'], axis=1)
    y_tr = y_tr.reset_index().drop(['index'], axis=1)
    y_t = y_t.reset_index().drop(['index'], axis=1)
    
    # соединяем исходный датасет и закодированный
    X_train_new = pd.concat([X_tr, X_train_onehot_df], axis=1) 
    X_test_new = pd.concat([X_t, X_test_onehot_df], axis=1) 
    
    # удаляем исходные столбцы
    X_train_new = X_train_new.drop(columns=list_col)
    X_test_new = X_test_new.drop(columns=list_col)
    
    return X_train_new, X_test_new, y_tr, y_t

In [21]:
X_c_train_ohe, X_c_valid_ohe, y_c_train, y_c_valid = onehotcod(
    X_c_train, X_c_valid, y_c_train, y_c_valid, columns_to_change)

In [22]:
lr_model = LinearRegression()
lr_model.fit(X_c_train_ohe, y_c_train)
y_c_train_pred = lr_model.predict(X_c_train_ohe)
y_c_valid_pred = lr_model.predict(X_c_valid_ohe)

print("Train R^2: {:.3f}".format(r2_score(y_c_train, y_c_train_pred)))
print("Test R^2: {:.3f}".format(r2_score(y_c_valid, y_c_valid_pred)))

Train R^2: 0.647
Test R^2: 0.693


In [38]:
# сделаем копии датасетов признаков
X_w = df.drop('price', axis=1)
y_w = df['price']

In [39]:
X_w_train, X_w_valid, y_w_train, y_w_valid = train_test_split(X_w, y_w, test_size=0.2, random_state=30)

In [40]:
# заполним пропуски по полю Погода среднем значением 
X_w_train['weather'] = X_w_train['weather'].fillna(np.round(np.mean(X_w_train['weather']), 2))
X_w_valid['weather'] = X_w_valid['weather'].fillna(np.round(np.mean(X_w_valid['weather']), 2))

In [41]:
X_w_train['drive'].value_counts(True).head(1)

4wd    0.738406
Name: drive, dtype: float64

In [42]:
X_w_valid['drive'].value_counts(True).head(1)

4wd    0.734922
Name: drive, dtype: float64

In [47]:
X_w_train['size'] = X_w_train['size'].fillna('full-size')
X_w_train['drive'] = X_w_train['drive'].fillna('4wd')
 
X_w_valid['size'] = X_w_valid['size'].fillna('full-size')
X_w_valid['drive'] = X_w_valid['drive'].fillna('4wd')

In [65]:
X_w_train_new, X_w_valid_new, y_w_train_new, y_w_valid_new = onehotcod(X_w_train, X_w_valid, y_w_train, y_w_valid, columns_to_change)

In [66]:
X_w_valid_new.shape

(1404, 27)

In [67]:
lr_model.fit(X_w_train_new, y_w_train_new)

y_w_train_pred = lr_model.predict(X_w_train_new)
y_w_valid_pred = lr_model.predict(X_w_valid_new)

print("Train R^2: {:.3f}".format(r2_score(y_w_train_new, y_w_train_pred)))
print("Test R^2: {:.3f}".format(r2_score(y_w_valid_new, y_w_valid_pred)))

Train R^2: 0.649
Test R^2: 0.465


#### РАБОТА С ВЫБРОСАМИ